In [1]:
import csv
from datetime import datetime
import numpy as np
import pandas as pd
import time 
from datetime import datetime
import os
from os import path
from glob import glob

import ijson
from tqdm import tqdm

# Read csv

In [2]:
# Feb data
zip_dir  = "../data/new_dfs_jst"
csvfiles = glob(os.path.join(zip_dir, 'dfor*'))

dfors = []
for csvfile in csvfiles:
    df = pd.read_csv(csvfile).drop('Unnamed: 0', axis=1)
    dfors.append(df)
    

KeyboardInterrupt: 

In [3]:
# Mar data
zip_dir  = "../data/new_dfs2_jst"
csvfiles = glob(os.path.join(zip_dir, 'dfor*'))

dfrts = []
for csvfile in csvfiles:
    df1 = pd.read_csv(csvfile).drop('Unnamed: 0', axis=1)
    dfrts.append(df1)
    

KeyboardInterrupt: 

In [4]:
dfors[0]

,user_id,date,tweet,RT_flag,year,month,day,hour,time
0,1227267163884384257,2020-03-31,勇作さんは影まで込みで(^O^)だと思っているところがあるから、影を描かないとソワソワする。,False,2020,3,31,23,23:59:58
1,1191941158181703681,2020-03-31,始めて園に通うこどもたちを迎える先生はきっと1日フルパワー全開の1日になると思います。保育が...,False,2020,3,31,23,23:59:58
2,778167031598374912,2020-03-31,なんかもうだるい自分が嫌,False,2020,3,31,23,23:59:58
3,825005837185748992,2020-03-31,こまるチャレンジいきます https://t.co/pdNmj8SqxW,False,2020,3,31,23,23:59:57
4,1635335509,2020-03-31,₍₍ ( * ॑ ॑*) ⁾⁾ｿﾜｿﾜ,False,2020,3,31,23,23:59:57
...,...,...,...,...,...,...,...,...,...
1114096,1163845866408402946,2020-03-28,@mikaze_ryo 見られてたら緊張するけど逆にフルコンできる,False,2020,3,28,21,21:36:00
1114097,1009407120549146625,2020-03-28,@waon_asuka 世界中コロナ蔓延していて、恐ろしいよね😱地球温暖化といい不安だらけな...,False,2020,3,28,21,21:36:00
1114098,1117088971316580352,2020-03-28,今の変にマセてる小学生じゃなくて、お互い気になるんだけどつい意地悪しちゃう！みたいな可愛い小...,False,2020,3,28,21,21:36:00
1114099,1128590746691325952,2020-03-28,@teterucameranko そそ、イベント系もそうやし現金給付に関してクレクレしてる奴...,False,2020,3,28,21,21:36:00


# Filter V4 by 'エロい' - Day

In [6]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def filterV4(dfors, filetype, savedir, key_lang, filt_flag):
    # Prepartion of dfw 
    results = []
    dfs = []
    dfw = pd.read_csv('../data/WordTimeSeries.csv', encoding='utf-8').rename(columns={'Unnamed: 0': 'types',
                                                                                    'file name': 'file_name', 
                                                                                    'orignal form': 'orignal_form',
                                                                                    'English translation': 'English_translation'})
    booleanDictionary = {True: 'TRUE', False: 'FALSE'} # Drop nan and False in order to use query 
    dfw = dfw.replace(booleanDictionary)
    dfw = dfw.dropna(how='all')
    
    # Prepration of keywords
    filenames = [filename for filename in dfw.file_name.tolist() if filetype in filename]
    xd = {}
    xd['T'] = 'file_name.str.contains("T")'
    xd['D'] = 'file_name.str.contains("D")'
    xd['A'] = 'file_name.str.contains("A")'
    xd['V'] = 'file_name.str.contains("V")'
    xd['F'] = 'file_name.str.contains("F")'
    xd['C'] = 'file_name.str.contains("C")'
    
    query = xd[filetype]
    dfw_c = dfw.query(query, engine='python')
    if key_lang == 'jp':
        keywords = dfw_c.orignal_form.tolist() # English.ver: keywords = dfw_c.English_translation.tolist() 
    elif key_lang == 'en':
        keywords = dfw_c.English_translation.tolist() 
    else:
        print('Put accurate keyword language')
            
    # Find tweets
    for i, dfor in tqdm(enumerate(dfors)):
         # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]
        
        # Add columns of keywords whose cell have 1 if this tweet includes a keyword 
        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
        
        d4 = dfor_c[dfor_c.V4 == 1]
        
        # Finally Connect dfs
        if i == 0:
            dfnew = d4
        else:
            dfnew = pd.concat([dfnew, d4], axis=0)
    
    
    # DF Only V4
    d4 = dfnew[dfnew.V4 == 1].drop(['V1', 'V2', 'V3', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20'], axis = 1)
    if filt_flag:
        # Remove 'エロい'
        d4_without_eroi = d4[~d4['tweet'].str.contains('エロい', na = False)]
        outname = 'V4b.csv'
    else:
        d4_without_eroi = d4
        outname = 'V4a.csv'

     # Make rows
    uni_dates = d4_without_eroi['date'].tolist()
    uni_dates = sorted(set(uni_dates), key=uni_dates.index) # date values
    rows = []
    for date in uni_dates:
        r = d4_without_eroi.groupby('date').get_group(date).V4.sum()
        rows.append(r)
    data = np.array([uni_dates, rows]).T
    # columns
    cols = ['date', 'V4']
    # Create DataFrame
    dft = pd.DataFrame(data, columns=cols).groupby('date').sum()
    
    # Save to csv
    outdir = savedir
    if not os.path.exists(outdir):
            os.makedirs(outdir)
    savename = os.path.join(outdir, outname)
    dft.to_csv(savename)
            
    return dft

In [8]:
# OR
filetypes = ['V']
for filetype in filetypes:
    d4_without_eroi = filterV4(dfors, filetype=filetype, savedir='../new_results_jst2/orjp/day', key_lang='jp', filt_flag=True)

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
10it [00:45,  4.54s/it]


In [7]:
# RT
filetypes = ['V']
for filetype in filetypes:
    d4_without_eroi = filterV4(dfrts, filetype=filetype, savedir='../new_results2/rtjp/day', key_lang='jp', filt_flag=True)

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
10it [00:31,  3.20s/it]


In [15]:
d4_without_eroi

,V4
date,
2020-01-31,568
2020-02-01,2336
2020-02-02,2898
2020-02-03,2461
2020-02-04,2492
2020-02-05,2403
2020-02-06,2268
2020-02-07,2005
2020-02-08,2171


# Filter V4 by 'エロい' - hour

In [14]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def findtwt_hour(dfors, filetype, savedir, key_lang, filt_flag):
    # Prepartion of dfw 
    results = []
    dfs = []
    dfw = pd.read_csv('../data/WordTimeSeries.csv', encoding='utf-8').rename(columns={'Unnamed: 0': 'types',
                                                                                    'file name': 'file_name', 
                                                                                    'orignal form': 'orignal_form',
                                                                                    'English translation': 'English_translation'})
    booleanDictionary = {True: 'TRUE', False: 'FALSE'} # Drop nan and False in order to use query 
    dfw = dfw.replace(booleanDictionary)
    dfw = dfw.dropna(how='all')

    # Prepration of keywords
    filenames = [filename for filename in dfw.file_name.tolist() if filetype in filename]
    xd = {}
    xd['T'] = 'file_name.str.contains("T")'
    xd['D'] = 'file_name.str.contains("D")'
    xd['A'] = 'file_name.str.contains("A")'
    xd['V'] = 'file_name.str.contains("V")'
    xd['F'] = 'file_name.str.contains("F")'
    xd['C'] = 'file_name.str.contains("C")'

    query = xd[filetype]
    dfw_c = dfw.query(query, engine='python')
    
    # Keyword 
    if key_lang == 'jp':
        keywords = dfw_c.orignal_form.tolist() # English.ver: keywords = dfw_c.English_translation.tolist() 
    elif key_lang == 'en':
        keywords = dfw_c.English_translation.tolist() 
    else:
        print('Put accurate keyword language')
    
    # Find tweets
    for i, dfor in tqdm(enumerate(dfors)):
        # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]

        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
#         dfs.append(dfor_c)
        
        d4 = dfor_c[dfor_c.V4 == 1]
        
        # Finally Connect dfs
        if i == 0:
            dfnew = d4
        else:
            dfnew = pd.concat([dfnew, d4], axis=0)
            

    # DF Only V4
    d4 = dfnew[dfnew.V4 == 1].drop(['V1', 'V2', 'V3', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20'], axis = 1)
    if filt_flag:
        # Remove 'エロい'
        d4_without_eroi = d4[~d4['tweet'].str.contains('エロい', na = False)]
        outname = 'V4b_hour.csv'
    else:
        d4_without_eroi = d4
        outname = 'V4a_hour.csv'
        
    # Create csv file by day
    all_df = []
    nextdf, nextdf_date = '', ''
    
    # Save dir
    outdir = savedir
    if not os.path.exists(outdir):
            os.makedirs(outdir)

#     for i, df in enumerate(dfs):
    df = d4_without_eroi
    unique_dates = df['date'].tolist()
    unique_dates = sorted(set(unique_dates), key=unique_dates.index) # date values
    df_date = df.groupby('date') # df grouped by date   

    for date in unique_dates:
        df_oneday = df_date.get_group(date) # df of one day
        dfh = df_oneday.groupby('hour').sum()
        dfh = dfh.drop(['year', 'month', 'user_id', 'RT_flag', 'day'],axis=1)

        # Save csv
        outname = filetype+str(date)+'.csv'
        savename = os.path.join(outdir, outname)

        if os.path.exists(savename):
            # File1
            existedfile = pd.read_csv(savename)
            # いったんcsvにする
            _saveonce = 'once.csv'
            dfh.to_csv(_saveonce)
            # File2 
            once = pd.read_csv(_saveonce)
            # Concat 
            df_oneday1 = pd.concat([existedfile, once], axis=0)
            # Finally save
            dfh = df_oneday1.groupby('hour').sum()

        # Save to csv
        dfh.to_csv(savename)


In [15]:
# Orjp
filetypes = ['V']
# filetypes = ['D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    findtwt_hour(dfors, filetype=filetype, savedir='testV4hour', key_lang='jp', filt_flag=True)

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
10it [00:40,  4.00s/it]


In [18]:
test = pd.read_csv('testV4hour/V2020-03-15.csv')
real = pd.read_csv('../new_results_jst2/orjp/day/V4b.csv')

In [31]:
real.loc[14, 'V4']

2985

In [32]:
test.V4.sum()

2985.0

In [35]:
path = '../new_results2/orjp/day/'
a = pd.read_csv(path+'V4a.csv').rename(columns={'V4': 'V4b'})
b = pd.read_csv(path+'V4b.csv').rename(columns={'V4': 'V4b'})

d = pd.concat([a, b.V4b], axis =1, join='inner')
d.diff(axis=1)

,date,V4b,V4b
0,NaN,NaN,0.0
1,NaN,NaN,-2.0
2,NaN,NaN,0.0
3,NaN,NaN,-2.0
4,NaN,NaN,-1.0
5,NaN,NaN,0.0
6,NaN,NaN,-1.0
7,NaN,NaN,-1.0
8,NaN,NaN,-2.0
9,NaN,NaN,-4.0


In [29]:
d = pd.concat([a, b.V4b], axis =1, join='inner')
d.diff(axis=1)

In [28]:
d.diff(axis=1)

,date,V4b,V4b
0,NaN,NaN,0.0
1,NaN,NaN,0.0
2,NaN,NaN,-2.0
3,NaN,NaN,0.0
4,NaN,NaN,0.0
5,NaN,NaN,0.0
6,NaN,NaN,0.0
7,NaN,NaN,0.0
8,NaN,NaN,0.0
9,NaN,NaN,0.0
